<a href="https://colab.research.google.com/github/ekity1002/DL4US_FINAL_WORK/blob/master/%E6%A5%BD%E5%A4%A9%E3%83%AC%E3%82%B7%E3%83%94%E5%8F%96%E5%BE%97.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# -*- coding:utf-8 -*-

import requests
import csv
from tqdm import tqdm
from bs4 import BeautifulSoup
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline


In [0]:

def write_tsv(path, rows):
    with open(path, 'a') as f:
        writer = csv.writer(f, delimiter='\t')
        for row in rows:
            writer.writerow(row)

def scrape(start_url, page_range):
    global N
    for i in tqdm(page_range):
        res = requests.get(f"{start_url}"+f"/{i}/")
        soup = BeautifulSoup(res.text, 'html.parser')

        # メニュー名のリンクリストを作成
        menu_links = [n for n in soup.select('.clearfix #recipe_link, #recipe_detail_link')]
        urls = [l['href'] for l in menu_links]

        # メニュー内容のスクレイピング
        for url in urls:
            # スクレイピング対象の URL にリクエストを送り HTML を取得する
            #print(url)
            res = requests.get('https://recipe.rakuten.co.jp/'+url)

            # レスポンスの HTML から BeautifulSoup オブジェクトを作る
            soup = BeautifulSoup(res.text, 'html.parser')

            # CSS セレクターを使って author を全て取得する
            # おつまみかどうか判定
            # author_names = [n.get_text(strip=True) for n in soup.select('a[sctag="recipe_detail_subcategory_medium"]')]
            # print(author_names)
            # not おつまみ continue

            # 画像
            imgs = [n for n in soup.select('img[itemprop="image"]')]
            if len(imgs)==0 :
                continue
            src = imgs[0]['src']
            r = requests.get(src)# title タグの文字列を取得する
            img_path = f'img{N}.jpg'
            with open(img_path, 'wb') as f:
                f.write(r.content)
            IMG_PATHS.append([img_path])
            
            # タイトル
            title_text = soup.find('h1').get_text(strip=True)
            #print(title_text.split()[0])

            # 材料
            zairyou = [n.get_text(strip=True) for n in soup.select('li[itemprop="ingredients"] a.name, div.name')]
            #print(zairyou)
            # カテゴリ(ビール/日本酒/ワイン/など)
            # category = [n.get_text(strip=True) for n in soup.select('a[sctag="recipe_detail_subcategory_small"]')]
            ZAIRYOU.append(zairyou)
            #print(ZAIRYOU)
            

            # 手順
            instructions = [n.get_text(strip=True) for n in soup.select('p.stepMemo')]
            #print(instructions)
            INSTRUCTIONS.append(instructions)

            N += 1
            #print("@@@@@@@@@@@@@@@@@@@@@@@@@")

# 訓練データ

In [10]:
# 一覧取得用
N = 1
ZAIRYOU = []
INSTRUCTIONS = []
IMG_PATHS = []
scrape('https://recipe.rakuten.co.jp/category/26-260-1068', range(1,51)) #ビールに合う
scrape('https://recipe.rakuten.co.jp/category/26-260-1069', range(1,51)) #ワイン
scrape('https://recipe.rakuten.co.jp/category/26-260-1070', range(1,51)) #日本酒
scrape('https://recipe.rakuten.co.jp/category/26-260-1071', range(1,51)) #焼酎
print(len(ZAIRYOU))
print(len(INSTRUCTIONS))
print(len(IMG_PATHS))



  2%|▏         | 1/50 [00:21<17:52, 21.88s/it]

  4%|▍         | 2/50 [00:49<18:46, 23.47s/it]

  6%|▌         | 3/50 [01:08<17:26, 22.27s/it]

  8%|▊         | 4/50 [01:28<16:37, 21.68s/it]

 10%|█         | 5/50 [01:48<15:44, 20.99s/it]

 12%|█▏        | 6/50 [02:07<15:00, 20.46s/it]

 14%|█▍        | 7/50 [02:24<13:55, 19.44s/it]

 16%|█▌        | 8/50 [02:44<13:39, 19.52s/it]

 18%|█▊        | 9/50 [03:09<14:32, 21.27s/it]

 20%|██        | 10/50 [03:27<13:31, 20.28s/it]

 22%|██▏       | 11/50 [03:50<13:41, 21.06s/it]

 24%|██▍       | 12/50 [04:13<13:44, 21.70s/it]

 26%|██▌       | 13/50 [04:39<14:04, 22.84s/it]

 28%|██▊       | 14/50 [05:05<14:19, 23.87s/it]

 30%|███       | 15/50 [05:26<13:30, 23.16s/it]

 32%|███▏      | 16/50 [05:50<13:10, 23.24s/it]

 34%|███▍      | 17/50 [06:14<12:53, 23.44s/it]

 36%|███▌      | 18/50 [06:35<12:13, 22.91s/it]

 38%|███▊      | 19/50 [06:58<11:47, 22.84s/it]

 40%|████      | 20/50 [07:22<11:31, 23.05s/it]

 42%|████▏     | 21/50 [07:

4000
4000
4000


In [0]:
# 保存
INST_PATH = '/content/drive/My Drive/Colab Notebooks/data/INSTRUCTIONS.tsv'
ZAIRYOU_PATH = '/content/drive/My Drive/Colab Notebooks/data/ZAIRYOU.tsv'
IMGTSV_PATH = '/content/drive/My Drive/Colab Notebooks/data/IMG_PATH.tsv'
write_tsv(INST_PATH, INSTRUCTIONS)
write_tsv(ZAIRYOU_PATH, ZAIRYOU)
write_tsv(IMGTSV_PATH, IMG_PATHS)

In [12]:
!zip train_img.zip *.jpg
!cp train_img.zip '/content/drive/My Drive/Colab Notebooks/data/'

  adding: img1000.jpg (deflated 0%)
  adding: img1001.jpg (deflated 0%)
  adding: img1002.jpg (deflated 0%)
  adding: img1003.jpg (deflated 0%)
  adding: img1004.jpg (deflated 1%)
  adding: img1005.jpg (deflated 0%)
  adding: img1006.jpg (deflated 0%)
  adding: img1007.jpg (deflated 0%)
  adding: img1008.jpg (deflated 0%)
  adding: img1009.jpg (deflated 0%)
  adding: img100.jpg (deflated 0%)
  adding: img1010.jpg (deflated 0%)
  adding: img1011.jpg (deflated 0%)
  adding: img1012.jpg (deflated 0%)
  adding: img1013.jpg (deflated 0%)
  adding: img1014.jpg (deflated 0%)
  adding: img1015.jpg (deflated 0%)
  adding: img1016.jpg (deflated 0%)
  adding: img1017.jpg (deflated 0%)
  adding: img1018.jpg (deflated 0%)
  adding: img1019.jpg (deflated 1%)
  adding: img101.jpg (deflated 1%)
  adding: img1020.jpg (deflated 0%)
  adding: img1021.jpg (deflated 0%)
  adding: img1022.jpg (deflated 1%)
  adding: img1023.jpg (deflated 1%)
  adding: img1024.jpg (deflated 0%)
  adding: img1025.jpg (deflate

#テストデータ

In [6]:
# 一覧取得用
N = 1
ZAIRYOU = []
INSTRUCTIONS = []
IMG_PATHS = []
scrape('https://recipe.rakuten.co.jp/category/26-260-1068', range(52,61)) #ビールに合う
scrape('https://recipe.rakuten.co.jp/category/26-260-1069', range(52,61)) #ビールに合う
scrape('https://recipe.rakuten.co.jp/category/26-260-1070', range(52,61)) #ビールに合う
scrape('https://recipe.rakuten.co.jp/category/26-260-1071', range(52,61)) #ビールに合う
print(len(ZAIRYOU))
print(len(INSTRUCTIONS))
print(len(IMG_PATHS))

100%|██████████| 9/9 [03:49<00:00, 26.48s/it]

720
720
720


In [0]:
# ファイル保存
INST_PATH = '/content/drive/My Drive/Colab Notebooks/data/test_INSTRUCTIONS.tsv'
ZAIRYOU_PATH = '/content/drive/My Drive/Colab Notebooks/data/test_ZAIRYOU.tsv'
IMGTSV_PATH = '/content/drive/My Drive/Colab Notebooks/data/test_IMG_PATH.tsv'
write_tsv(INST_PATH, INSTRUCTIONS)
write_tsv(ZAIRYOU_PATH, ZAIRYOU)
write_tsv(IMGTSV_PATH, IMG_PATHS)

In [7]:
!zip test_img.zip *.jpg
!cp test_img.zip '/content/drive/My Drive/Colab Notebooks/data/'

  adding: img100.jpg (deflated 1%)
  adding: img101.jpg (deflated 1%)
  adding: img102.jpg (deflated 1%)
  adding: img103.jpg (deflated 1%)
  adding: img104.jpg (deflated 1%)
  adding: img105.jpg (deflated 1%)
  adding: img106.jpg (deflated 1%)
  adding: img107.jpg (deflated 1%)
  adding: img108.jpg (deflated 1%)
  adding: img109.jpg (deflated 1%)
  adding: img10.jpg (deflated 1%)
  adding: img110.jpg (deflated 1%)
  adding: img111.jpg (deflated 1%)
  adding: img112.jpg (deflated 1%)
  adding: img113.jpg (deflated 1%)
  adding: img114.jpg (deflated 1%)
  adding: img115.jpg (deflated 1%)
  adding: img116.jpg (deflated 1%)
  adding: img117.jpg (deflated 1%)
  adding: img118.jpg (deflated 1%)
  adding: img119.jpg (deflated 1%)
  adding: img11.jpg (deflated 1%)
  adding: img120.jpg (deflated 2%)
  adding: img121.jpg (deflated 2%)
  adding: img122.jpg (deflated 1%)
  adding: img123.jpg (deflated 1%)
  adding: img124.jpg (deflated 1%)
  adding: img125.jpg (deflated 1%)
  adding: img126.jpg (